## ERP SCANR


.

In [1]:
## Imports

# Import custom code
from erpsc import *

# TEST IMPORTS
import requests
from bs4 import BeautifulSoup

In [2]:
## Things to search through - full set
erps = ['P50', 'P100', 'P200', 'P300', 'P3a', 'P3b', 'P600', 'N100', 'N170', 'N200',\
         'N270', 'N2pc', 'N400', 'MMN', 'LPC', 'CNV', 'ERN', 'ELAN', 'CPS', 'LRP', 'LDN']
cogs = ['language', 'memory', 'attention', 'motor', 'decision making', 'vision', \
         'auditory', 'emotion', 'categorization', 'reward', 'spatial']

In [4]:
# Small test set of words
erps = ['P50', 'P100']
cogs = ['language', 'memory']

## Co-Occurence of Term Analysis

.

In [5]:
# Initialize object for term count co-occurences. 
term_counts = ERPSC_Count()

# Set erp and cog lists as terms to use
term_counts.set_erps(erps)
term_counts.set_cogs(cogs)

In [6]:
# Scrape the co-occurence of terms data
term_counts.scrape_data()

In [7]:
# Check the most commonly associated COG term for each ERP
term_counts.check_erps()

For the  P50   the most common association is 	 memory     with 	 %01.39
For the  P100  the most common association is 	 memory     with 	 %02.44


In [8]:
# Check the most commonly associated ERP term for each COG
term_counts.check_cogs()

For  language             the strongest associated ERP is 	 P100  with 	 %00.60
For  memory               the strongest associated ERP is 	 P100  with 	 %02.44


In [9]:
# Check the terms with the most papers
term_counts.check_top()

The most studied ERP is  P50     with     8177 papers
The most studied COG is  memory  with   213047  papers


In [10]:
# Check how many papers were found for each term - ERPs
term_counts.check_counts('erp')

P50   -     8177
P100  -     2171


In [11]:
# Check how many papers were found for each term - COGs
term_counts.check_counts('cog')

language           -     135605
memory             -     213047


In [12]:
# Save pickle file of results
term_counts.save_pickle()

In [13]:
# Test load pickle
aa = load_pickle_counts()

In [14]:
aa.check_counts('cog')

language           -     135605
memory             -     213047


## WORD ANALYSIS

.

In [ ]:
test = ERPSC_Words()

In [40]:
# Test Code

#page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“N270”AND”Language”')
page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&field=word&term=“P50”&retmax=50')

page_soup = BeautifulSoup(page.content)

counts = page_soup.find_all('count')

for i in range(0, len(counts)):
    count = counts[i]
    ext = count.text
    print int(ext)

8177
8177


In [41]:
page_soup

<!DOCTYPE eSearchResult PUBLIC "-//NLM//DTD esearch 20060628//EN" "http://eutils.ncbi.nlm.nih.gov/eutils/dtd/20060628/esearch.dtd">
<html><body><esearchresult><count>8177</count><retmax>50</retmax><retstart>0</retstart><idlist>
<id>27315458</id>
<id>27308511</id>
<id>27302421</id>
<id>27295376</id>
<id>27290699</id>
<id>27287810</id>
<id>27281270</id>
<id>27279585</id>
<id>27271273</id>
<id>27268961</id>
<id>27264783</id>
<id>27264614</id>
<id>27262063</id>
<id>27256734</id>
<id>27256730</id>
<id>27252578</id>
<id>27250735</id>
<id>27243897</id>
<id>27242078</id>
<id>27242072</id>
<id>27238796</id>
<id>27234131</id>
<id>27217271</id>
<id>27216047</id>
<id>27198178</id>
<id>27197825</id>
<id>27197812</id>
<id>27194765</id>
<id>27190325</id>
<id>27183665</id>
<id>27180104</id>
<id>27178864</id>
<id>27178167</id>
<id>27176922</id>
<id>27168850</id>
<id>27168061</id>
<id>27162449</id>
<id>27161000</id>
<id>27147748</id>
<id>27145282</id>
<id>27141897</id>
<id>27139571</id>
<id>27138602</id

In [42]:
ids = page_soup.find_all('id')

In [143]:
print(ids)
len(ids)

[<id>27315458</id>, <id>27308511</id>, <id>27302421</id>, <id>27295376</id>, <id>27290699</id>, <id>27287810</id>, <id>27281270</id>, <id>27279585</id>, <id>27271273</id>, <id>27268961</id>, <id>27264783</id>, <id>27264614</id>, <id>27262063</id>, <id>27256734</id>, <id>27256730</id>, <id>27252578</id>, <id>27250735</id>, <id>27243897</id>, <id>27242078</id>, <id>27242072</id>, <id>27238796</id>, <id>27234131</id>, <id>27217271</id>, <id>27216047</id>, <id>27198178</id>, <id>27197825</id>, <id>27197812</id>, <id>27194765</id>, <id>27190325</id>, <id>27183665</id>, <id>27180104</id>, <id>27178864</id>, <id>27178167</id>, <id>27176922</id>, <id>27168850</id>, <id>27168061</id>, <id>27162449</id>, <id>27161000</id>, <id>27147748</id>, <id>27145282</id>, <id>27141897</id>, <id>27139571</id>, <id>27138602</id>, <id>27132515</id>, <id>27126340</id>, <id>27122325</id>, <id>27118659</id>, <id>27108556</id>, <id>27101794</id>, <id>27098223</id>]


50

In [150]:
aa = str(ids[0].text)
print aa

for i in range(1, 5):
    aa = aa + ',' + ids[i].text
    
print aa

27315458
27315458,27308511,27302421,27295376,27290699


In [156]:
qq = ids_to_str(ids)
print(qq)

27315458,27308511,27302421,27295376,27290699,27287810,27281270,27279585,27271273,27268961,27264783,27264614,27262063,27256734,27256730,27252578,27250735,27243897,27242078,27242072,27238796,27234131,27217271,27216047,27198178,27197825,27197812,27194765,27190325,27183665,27180104,27178864,27178167,27176922,27168850,27168061,27162449,27161000,27147748,27145282,27141897,27139571,27138602,27132515,27126340,27122325,27118659,27108556,27101794,27098223


In [99]:
art_page = requests.get('http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&retmode=xml&id=' + ids[0].text + ',' + ids[1].text)

art_page_soup = BeautifulSoup(art_page.content, "xml")

In [100]:
art_page_soup

<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE PubmedArticleSet PUBLIC "-//NLM//DTD PubMedArticle, 1st January 2016//EN" "http://www.ncbi.nlm.nih.gov/corehtml/query/DTD/pubmed_160101.dtd">
<PubmedArticleSet>
<PubmedArticle>
<MedlineCitation Owner="NLM" Status="Publisher">
<PMID Version="1">27315458</PMID>
<DateCreated>
<Year>2016</Year>
<Month>6</Month>
<Day>17</Day>
</DateCreated>
<DateRevised>
<Year>2016</Year>
<Month>6</Month>
<Day>17</Day>
</DateRevised>
<Article PubModel="Print-Electronic">
<Journal>
<ISSN IssnType="Electronic">1872-7123</ISSN>
<JournalIssue CitedMedium="Internet">
<Volume>242</Volume>
<PubDate>
<Year>2016</Year>
<Month>Apr</Month>
<Day>14</Day>
</PubDate>
</JournalIssue>
<Title>Psychiatry research</Title>
<ISOAbbreviation>Psychiatry Res</ISOAbbreviation>
</Journal>
<ArticleTitle>Association of catechol-O-methyltransferase Val(108/158) Met genetic polymorphism with schizophrenia, P50 sensory gating, and negative symptoms in a Chinese population.</ArticleTitle>
<

In [141]:
nArticles = len(art_page_soup.findAll('PubmedArticle'))
print(nArticles)

2


In [130]:
title = art_page_soup.findAll('ArticleTitle')
#print(len(title))
#print(title)

abstract_text = art_page_soup.findAll('AbstractText')
#print(len(abstract_text))
#print(abstract_text)

years = art_page_soup.findAll('DateCreated')
#print(len(years))
#print(years)

#print(type(years))
#art_page_soup.findChild?

aa = art_page_soup.findChild('DateCreated').find('Year')
#print(aa)
aa.find('Year')

bb = art_page_soup.findChildren('DateCreated')
bb[0].find('Year')
print(bb)

print(type(aa))
print(type(bb))

print(len(bb))


[<DateCreated>
<Year>2016</Year>
<Month>6</Month>
<Day>17</Day>
</DateCreated>, <DateCreated>
<Year>2016</Year>
<Month>06</Month>
<Day>16</Day>
</DateCreated>]
<class 'bs4.element.Tag'>
<class 'bs4.element.ResultSet'>
2


<Year>2016</Year>

In [34]:
#page_1 = requests.get('http://www.ncbi.nlm.nih.gov/pubmed/' + aa[0].text)
#page_1_soup = BeautifulSoup(page_1.content)

In [35]:
#page_1_soup

In [31]:
title = page_1_soup.find('title')
print title

abtract = page_1_soup.find('abstracttext')

<title>Association of catechol-O-methyltransferase Val108/158 Met genetic polymorphism with schizophrenia, P50 sensory gating, and negative symptoms in a ...  - PubMed - NCBI
</title>


In [32]:
aa = page_1_soup.find_all('h1')

In [33]:
aa

[<h1 class="img_logo"><a class="pmlogo offscreen" href="/pubmed" title="PubMed">PubMed</a></h1>,
 <h1>Association of catechol-O-methyltransferase Val108/158 Met genetic polymorphism with schizophrenia, P50 sensory gating, and negative symptoms in a Chinese population.</h1>]